In [2]:
#CLSP-Rmin with back-ordering example
from gurobipy import *
import time
import math

#start = time.time()

n = Model("CLSP-RminBO")

#Parameters
d= [[258, 248, 193, 213, 238, 276, 329, 359, 375, 362],
    [341, 322, 251, 288, 330, 376, 443, 476, 498, 471],
    [510, 473, 390, 422, 482, 570, 667, 712, 753, 721],
    [508, 480, 390, 418, 478, 580, 653, 726, 744, 712]]
f=  [110, 120, 100, 120]
#f= [165,180,150,180] #for f +50%
h= [1.5, 2, 2.5, 1.25]
hR= [1.5, 2, 2.5, 1.25]
#hR=[0.75,1,1.25,0.625] #for hR -50%
#hR=[1.125,1.5,1.875,0.9375] #for hR -25%
tp= [1, 1, 1, 1]
tpR= [0.75, 0.75, 0.75, 0.75]
#ts= [25, 20, 20, 15]
ts=[18.75,15,15,11.25] #for ts -25%
tsR= [25, 20, 20, 15]
#tsR=[18.75,15,15,11.25] #for tsR -25%
b= [2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000]
#b = [1880, 1880, 1880, 1880, 1880, 1880, 1880, 1880, 1880, 1880] #for b-6%
#b= [1860, 1860, 1860, 1860, 1860, 1860, 1860, 1860, 1860, 1860] #for b-7%
k= [0, 0, 0, 0]
M= 100000
s0 = [0,0,0,0]
sR0 = [0,0,0,0]
T=len(b)
print(T)
m=2
BO0= [0,0,0,0]

#Indices/Range of products and periods
Pro= range(len(f))
Pe = range(len(b))
print(Pe)
O=[0.01 for t in Pe] #print(round(random.uniform(0.0001,0.03), 3))


def compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBo):
    #Decision variables default settings are 0.0 lb for all continous variables (positive values)
    x = n.addVars(Pro, Pe, vtype=GRB.BINARY, name="x")
    s = n.addVars(Pro, Pe, vtype=GRB.CONTINUOUS, name="s")
    y = n.addVars(Pro, Pe, vtype=GRB.INTEGER, name="y")
    yS = n.addVars(Pro, Pe, vtype=GRB.INTEGER, name="yS")
    yR = n.addVars(Pro, Pe, vtype=GRB.INTEGER, name="yR")
    sR = n.addVars(Pro, Pe, vtype=GRB.CONTINUOUS, name="sR")
    xR = n.addVars(Pro, Pe, vtype=GRB.BINARY, name="xR")
    R = n.addVars(Pro, Pe, vtype=GRB.INTEGER, name="R")
    BO = n.addVars(Pro, Pe, vtype=GRB.CONTINUOUS, name="BO")
    #n.update()

    #Objective
    obj= quicksum(((f[j]*x[j,t]+h[j]*s[j,t])+(f[j]*xR[j,t]+hR[j]*sR[j,t])+((h[j]*m)*BO[j,t])) for j in Pro for t in Pe)
    n.setObjective(obj, GRB.MINIMIZE) 
    #n.update()

    #initialized back order variable NO NEED FOR CODING THIS CUS PYTHON INDEX 0 REFERS TO PERIOD 1
    #for j in Pro:
    #    if t==0:
    #        n.addConstr(BO[j,0]==BO0[j])
    #    else:
    #        n.addConstr(BO[j,t]==BO[j,t])
    #n.addConstrs((BO[j,0]==BO0[j]) for j in Pro) BO DOESNT NEED TO BE 0 AT THE END OF 1ST PERIOD

    #n.update()
    #s[j,T] the last period of product j equals 0
    n.addConstrs((s[j,T-1] == 0) for j in Pro)    #T is defined as T=len(b) to count how many periods we have =10                 
    n.addConstrs((sR[j,T-1] == 0) for j in Pro)
    if CostBo==True:
        n.addConstrs((BO[j,T-1] == 0) for j in Pro)
    else:
        pass
    #Inventory balance constraints
    for j in Pro:
        for t in Pe:
            if t==0:
                n.addConstr(s[j,t]-BO[j,t]==s0[j]+yS[j,t]+yR[j,t]-d[j][t]-BO0[j])
            else:
                n.addConstr(s[j,t]-BO[j,t]==s[j,t-1]+yS[j,t]+yR[j,t]-d[j][t]-BO[j,t-1])
    #n.update()
    for j in Pro:
        for t in Pe:
            if t==0:
                n.addConstr(sR[j,t]==sR0[j]+R[j,t]-yR[j,t])
            else:
                n.addConstr(sR[j,t]==sR[j,t-1]+R[j,t]-yR[j,t])


    #Capacity constraints
    n.addConstrs(
        (quicksum(((tp[j]*y[j,t]+ts[j]*x[j,t])+(tpR[j]*yR[j,t]+tsR[j]*xR[j,t])) for j in Pro) <= b[t]
            for t in Pe), "Capacity-Ct")

    #Big M+ Big M for rework #change from d[j][t:] to d[j][:]
    n.addConstrs((y[j,t]<= sum(d[j][:])*x[j,t] for j in Pro for t in Pe), "BigM-Ct")
    n.addConstrs((yR[j,t]<= sum(d[j][:])*xR[j,t] for j in Pro for t in Pe), "BigM-Ct for Rework")

    #!production amount at least as size of lot size
    for j in Pro:
        for t in Pe:
            n.addConstr(y[j,t]>=k[j]*x[j,t])
            #floor brackets  
            #m.addConstr(yS[j,t] ==math.floor((y[j,t]*(1-O[t])))) # this doesn't work with Gurobi, so use two constraints below
            n.addConstr(yS[j,t] >= (y[j,t]*(1-O[t]) - 0.9999))
            n.addConstr(yS[j,t] <= (y[j,t]*(1-O[t])))
            #ceil brackets
            #m.addConstr(R[j,t]==math.ceil((y[j,t]*O[t]))) # same here
            n.addConstr(R[j,t]>=(y[j,t]*O[t]))
            n.addConstr(R[j,t]<=(y[j,t]*O[t] + 0.9999))

    #!production amount at least as size of lot size
    for j in Pro:
        for t in Pe:
            n.addConstr(y[j,t]>=k[j]*x[j,t])
            #floor brackets  
            #m.addConstr(yS[j,t] ==math.floor((y[j,t]*(1-O[t])))) # this doesn't work with Gurobi, so use two constraints below
            n.addConstr(yS[j,t] >= (y[j,t]*(1-O[t]) - 0.9999))
            n.addConstr(yS[j,t] <= (y[j,t]*(1-O[t])))
            #ceil brackets
            #m.addConstr(R[j,t]==math.ceil((y[j,t]*O[t]))) # same here
            n.addConstr(R[j,t]>=(y[j,t]*O[t]))
            n.addConstr(R[j,t]<=(y[j,t]*O[t] + 0.9999))

            
    #Set time limit
    #n.Params.timeLimit =60.0
    #logging.basicConfig(filename='info.log', level=logging.INFO)

    n.optimize()

    n.printAttr('x','y*')
    n.printAttr('x','sR*')
    #n.update()

    #PRINT ALL RESULTS BELOW
    #holding cost
    pp=0
    for j in Pro:
        for t in Pe:
            pp= pp + (h[j]*s[j,t])
    #holding cost for rework
    ppp=0
    for j in Pro:
        for t in Pe:
            ppp= ppp + (hR[j]*sR[j,t])
    #setup cost combined
    pppp=0
    for j in Pro:
        for t in Pe:
            pppp=pppp+(f[j]*x[j,t]+f[j]*xR[j,t])
    #setup cost for production
    fcost=0
    for j in Pro:
        for t in Pe:
            fcost=fcost+(f[j]*x[j,t])
    #Nr of setups for production
    xprod=0
    for j in Pro:
        for t in Pe:
            xprod += x[j,t].x
    #Setup costs for rework
    fcostr=0
    for j in Pro:
        for t in Pe:
            fcostr=fcostr+(f[j]*xR[j,t])
    xxx=0
    for j in Pro:
        for t in Pe:
            xxx=xxx+(((tp[j]*y[j,t]+ts[j]*x[j,t])+(tpR[j]*yR[j,t]+tsR[j]*xR[j,t]))/b[j])*0.1
            
    #Nr of setups for rework
    xrework=0
    for j in Pro:
        for t in Pe:
            xrework += xR[j,t].x
    #Back order costs
    cbcost=0
    for j in Pro:
        for t in Pe:
            cbcost=cbcost+(h[j]*m)*BO[j,t].x
    cos=[]# BO0
    for j in Pro:
        cos.append(BO[j,Rp-1].x)
        
    mag=[] #s0
    for j in Pro:
        mag.append(s[j,Rp-1].x)
    
    magr=[] #sR0
    for j in Pro:
        magr.append(sR[j,Rp-1].x)
        
    #Objective fuction
    ZZ=n.ObjVal
    #Cost reduction
    #cred=0
    #for j in Pro:
    #    for t in Pe:
    #        cred=((ZZ*100)/(ZZ+cbcost))     


    print(pp.getValue(), "Table 4 h Holding cost")
    print(ppp.getValue(), "Table 4 hR Holding cost for rework")
    print(pppp.getValue(), "Setup cost combined")
    print(fcost.getValue(), "Table 4 f Setup cost for production")
    print(xprod, "Table 4 x Nr of set-ups for production")
    print(fcostr.getValue(), "Table 4 f Setup cost for rework")
    print(xrework, "Table 4 xR Setup cost for rework")
    print(xxx.getValue(), "Table 4 Capacity Utilization")
    print(ZZ, "Table 4 Z objective function value")
    print(cbcost,"Table 4 Back order costs cb")
    #print(cred,"Table 4 Cost reduction")
    #print("Total time:", time.time()-start)
    
    return cos, mag, magr
T=6    
Pe=range(T)    
Rp=2

for i in range(4):  
    if i ==0:
        print('step',i+1)
        CostBO= True
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO)
    
    elif i==3: #and T >len(b) and Rp<len(b):
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        print(BO0)
        print(s0)
        print(sR0)
        print(b)
        print(d)
        Pe=range(len(b))
        T=len(b)
        m=m*100 # backorder cost increased to recompensate no backorders in the last period 
        CostBO=False
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO)
    
    elif i==4: #T>len(b) and Rp>len(b)
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        Pe=range(len(b))
        T=len(b)
        Rp=len(b)
        m=m*1000
        print(BO0)
        print(s0)
        print(sR0)
        print(b)
        print(d)
        CostBO=False
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO)
    
    elif i<3:
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        print(i)
        print(len(b))
        print(b)
        if i==2:
            m=m*10
            CostBO=False
        else:
            CostBO=True
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO)

    else:
        print('end')


10
range(0, 10)
step 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 354 rows, 216 columns and 864 nonzeros
Model fingerprint: 0x8757cc42
Variable types: 72 continuous, 144 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 160 rows and 16 columns
Presolve time: 0.00s
Presolved: 194 rows, 200 columns, 648 nonzeros
Variable types: 40 continuous, 160 integer (48 binary)

Root relaxation: objective 7.040642e+02, 140 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  704.06424    0  116          -  

     0     0 4867.79962    0  119 5516.25000 4867.79962  11.8%     -    0s
     0     0 5049.48082    0  109 5516.25000 5049.48082  8.46%     -    0s
     0     0 5058.98815    0  111 5516.25000 5058.98815  8.29%     -    0s
     0     0 5064.71288    0  112 5516.25000 5064.71288  8.19%     -    0s
     0     0 5069.29989    0  113 5516.25000 5069.29989  8.10%     -    0s
     0     0 5153.82055    0  103 5516.25000 5153.82055  6.57%     -    0s
     0     0 5166.71457    0  106 5516.25000 5166.71457  6.34%     -    0s
     0     0 5174.71272    0  107 5516.25000 5174.71272  6.19%     -    0s
     0     0 5174.73494    0  110 5516.25000 5174.73494  6.19%     -    0s
H    0     0                    5507.2500000 5174.73494  6.04%     -    0s
     0     0 5184.67112    0  122 5507.25000 5184.67112  5.86%     -    0s
     0     0 5185.04325    0  126 5507.25000 5185.04325  5.85%     -    0s
     0     0 5188.15354    0  132 5507.25000 5188.15354  5.79%     -    0s
     0     0 5188.15354  

H  466   221                    35021.250000 34853.6028  0.48%   3.1    0s
H  706   275                    34998.750000 34937.5263  0.17%   2.6    0s

Cutting planes:
  Gomory: 8
  MIR: 22
  StrongCG: 3
  Zero half: 5

Explored 3236 nodes (9025 simplex iterations) in 1.46 seconds (0.42 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 34998.7 35021.3 35034 ... 46076.5

Optimal solution found (tolerance 1.00e-04)
Best objective 3.499875000000e+04, best bound 3.499858242862e+04, gap 0.0005%

    Variable            x 
-------------------------
      y[0,4]           26 
      y[0,5]         1400 
      y[1,1]         1377 
      y[1,2]           32 
      y[1,5]          499 
      y[2,2]          897 
      y[2,3]         1950 
      y[3,0]         1987 
      y[3,2]          867 
     yS[0,4]           25 
     yS[0,5]         1386 
     yS[1,1]         1363 
     yS[1,2]           31 
     yS[1,5]          494 
     yS[2,2]          888 
     yS[2,3]      

  Implied bound: 4
  MIR: 33
  StrongCG: 4
  Zero half: 17
  Relax-and-lift: 1

Explored 1 nodes (1148 simplex iterations) in 0.96 seconds (0.21 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 2.89837e+06 2.90337e+06 2.90388e+06 ... 7.26536e+07

Optimal solution found (tolerance 1.00e-04)
Best objective 2.898368250000e+06, best bound 2.898368250000e+06, gap 0.0000%

    Variable            x 
-------------------------
      y[0,2]         1426 
      y[1,0]         1901 
      y[1,2]            7 
      y[2,1]          938 
      y[2,4]         1909 
      y[3,3]          932 
      y[3,5]         1922 
     yS[0,2]         1411 
     yS[1,0]         1881 
     yS[1,2]            6 
     yS[2,1]          928 
     yS[2,4]         1889 
     yS[3,3]          922 
     yS[3,5]         1902 
     yR[0,5]           15 
     yR[1,0]           20 
     yR[1,3]            1 
     yR[2,1]           10 
     yR[2,4]           20 
     yR[3,4]           10 
     yR